<a href="https://colab.research.google.com/github/allan-jt/IJEPA-Thermal-Benchmark/blob/notebook/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preliminaries



In [1]:
!pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 898.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 35.3 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.3
    Uninstalling tokenizers-0.20.3:
      Successfully uninstalled tokenizers-0.20.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.46.3
    Uninstalling transformers-4.46.3:
      Successfully uninstalled transformers-4.46.3


In [2]:
# Imports required to run the demo code here:
# https://huggingface.co/onnx-community/ijepa_vith14_22k

import requests
from PIL import Image
from torch.nn.functional import cosine_similarity
from transformers import AutoModel, AutoProcessor

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

# Basic test of I-JEPA's context encoder

In [6]:
model_id = "facebook/ijepa_vith14_22k"
processor = AutoProcessor.from_pretrained(model_id)
model = AutoModel.from_pretrained(model_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/325 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/497 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.52G [00:00<?, ?B/s]

In [7]:
url_1 = "http://images.cocodataset.org/val2017/000000039769.jpg"
url_2 = "http://images.cocodataset.org/val2017/000000219578.jpg"
image_1 = Image.open(requests.get(url_1, stream=True).raw)
image_2 = Image.open(requests.get(url_2, stream=True).raw)

In [17]:
img_processed = processor(image_1, return_tensors="pt")
output = model(**img_processed)
# print(output.pooler_output.shape)

In [12]:
print(output.last_hidden_state[:, 0, :].shape)

torch.Size([1, 1280])


In [ ]:
def infer(image):
    inputs = processor(image, return_tensors="pt")
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1)

In [ ]:
embed_1 = infer(image_1)
embed_2 = infer(image_2)

In [ ]:
similarity = cosine_similarity(embed_1, embed_2)
print(similarity)

tensor([0.5071], grad_fn=<SumBackward1>)


# Loading Backbone

In [25]:
class ClassificationHead(nn.Module):
    def __init__(self, input_dim, hidden_dims, output_dim, dropout_rate=0.5, activation=nn.ReLU):
        super(ClassificationHead, self).__init__()

        self.input_dim = input_dim
        self.hidden_dims = hidden_dims
        self.num_classes = output_dim
        self.dropout_rate = dropout_rate

        layers = []
        dims = [input_dim] + hidden_dims

        for i in range(len(dims) - 1):
            layers.append(nn.Linear(dims[i], dims[i+1]))
            layers.append(nn.LayerNorm(dims[i+1]))
            layers.append(activation())
            layers.append(nn.Dropout(dropout_rate))

        # Add the final output layer
        layers.append(nn.Linear(dims[-1], self.num_classes))

        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

In [26]:
class TransformerWithClassificationHead(nn.Module):
    def __init__(self, vit, hidden_dims, output_dim, dropout_rate=0.5, activation=nn.ReLU, withLayerNorm=False):
        super(TransformerWithClassificationHead, self).__init__()
        self.vit = vit
        self.layer_norm = None
        if withLayerNorm:
            self.layer_norm = nn.LayerNorm(self.vit.config.hidden_size)
        self.classification_head = ClassificationHead(
            self.vit.config.hidden_size,
            hidden_dims,
            output_dim,
            dropout_rate,
            activation
        )

    def forward(self, pixel_values):
        outputs = self.vit(pixel_values)
        cls_token = outputs.last_hidden_state[:, 0, :]
        if self.layer_norm is not None:
          cls_token = self.layer_norm(cls_token)
        return self.classification_head(cls_token)

In [27]:
# Test
model_with_classifier = TransformerWithClassificationHead(
    model,
    [10],
    3,
    dropout_rate=0.5,
    activation=nn.ReLU,
    withLayerNorm=False
)

In [28]:
output = model_with_classifier(**img_processed)
print(output)

tensor([[-0.4896,  0.5661,  0.6515]], grad_fn=<AddmmBackward0>)
